# Find the best Preprocessing Steps During Model Selection for Stock Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2014-01-02,3.95,3.95,3.98,3.84,3.85,20548400
2014-01-03,4.00,4.00,4.00,3.88,3.98,22887200
2014-01-06,4.13,4.13,4.18,3.99,4.01,42398300
2014-01-07,4.18,4.18,4.25,4.11,4.19,42932100
2014-01-08,4.18,4.18,4.26,4.14,4.23,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()

In [4]:
dataset.head()

,Adj Close,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2014-01-03,4.00,4.00,4.00,3.88,3.98,22887200,1,1,1,0.012658
2014-01-06,4.13,4.13,4.18,3.99,4.01,42398300,1,1,1,0.032500
2014-01-07,4.18,4.18,4.25,4.11,4.19,42932100,0,1,0,0.012106
2014-01-08,4.18,4.18,4.26,4.14,4.23,30678700,0,0,0,0.000000
2014-01-09,4.09,4.09,4.23,4.05,4.20,30667600,0,0,1,-0.021531


In [5]:
dataset.tail()

,Adj Close,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2018-08-20,19.980000,19.980000,20.08,19.350000,19.790001,62983200,0,1,1,0.010622
2018-08-21,20.400000,20.400000,20.42,19.860001,19.980000,55629000,1,1,1,0.021021
2018-08-22,20.900000,20.900000,20.92,20.209999,20.280001,62002700,1,1,1,0.024510
2018-08-23,22.290001,22.290001,22.32,21.139999,21.190001,113444100,1,1,1,0.066507
2018-08-24,23.980000,23.980000,24.00,22.670000,22.910000,164328200,0,0,0,0.075819


In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [7]:
X = dataset.drop(["Buy_Sell"], axis=1)
y = dataset["Buy_Sell"]

In [8]:
# Create a combined preprocessing object
preprocess = FeatureUnion([('pca', PCA()), ("kbest", SelectKBest(k=1))])

In [9]:
# Create a pipeline
pipe = Pipeline([('preprocess', preprocess), ('classifier', LogisticRegression())])

In [10]:
# Create space of candidate values
search_space = [{'preprocess__pca__n_components': [1, 2, 3],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(0, 4, 10)}]

In [11]:
# Create grid search 
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)

In [12]:
# Fit grid search
best_model = clf.fit(X, y)

In [13]:
# View best hyperparameters
print('Best Number Of Princpal Components:', best_model.best_estimator_.get_params()['preprocess__pca__n_components'])
print('Best Penalty:', best_model.best_estimator_.get_params()['classifier__penalty'])
print('Best C:', best_model.best_estimator_.get_params()['classifier__C'])

Best Number Of Princpal Components: 1
Best Penalty: l1
Best C: 1.0
